This notebook will pull from the FPL API the weekly results for each player and stack all the results into a single df

Update weekly

In [9]:
import pandas as pd
import numpy as np
import requests

In [10]:
top500 = pd.read_csv('top500.csv', index_col='id_')
top500.columns

Index(['assists', 'bonus', 'bps', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'clean_sheets', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'creativity', 'dreamteam_count',
       'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'goals_conceded', 'goals_scored', 'ict_index', 'in_dreamteam',
       'influence', 'minutes', 'news', 'news_added', 'now_cost', 'own_goals',
       'penalties_missed', 'penalties_saved', 'photo', 'points_per_game',
       'red_cards', 'saves', 'second_name', 'selected_by_percent', 'special',
       'squad_number', 'status', 'team', 'team_code', 'threat', 'total_points',
       'transfers_in', 'transfers_in_event', 'transfers_out',
       'transfers_out_event', 'value_form', 'value_season', 'web_name',
       'yellow_cards'],
      dtype='object')

In [11]:
histories = {}

ids = top500.index

for id_ in ids:
    url = 'https://fantasy.premierleague.com/api/element-summary/' + str(id_) + '/'
    r = requests.get(url)
    player_data = r.json()
    player_history = player_data['history']
    
    histories[id_] = player_history

In [12]:
history_dfs = {}

for key in histories:
    df = pd.DataFrame(histories[key])
    df['gameweek'] = [i+1 for i in range((len(df.index)))]
    df['id_'] = key
    df['sum_points'] = [sum(df['total_points'].iloc[:i+1]) for i in range(len(df.index))]
    df['web_name'] = top500.loc[key]['web_name']
    df['next_week_points'] = df['total_points'].drop(0).append(pd.Series(0)).reset_index(drop=True)

    history_dfs[key] = df
    
    
    

In [13]:
history_dfs[215][['web_name', 'total_points', 'next_week_points']]

,web_name,total_points,next_week_points
0,De Bruyne,7,11
1,De Bruyne,11,5
2,De Bruyne,5,13
3,De Bruyne,13,1
4,De Bruyne,1,17
5,De Bruyne,17,9
6,De Bruyne,9,0
7,De Bruyne,0,3
8,De Bruyne,3,6
9,De Bruyne,6,2


In [6]:
top500histories = pd.concat(history_dfs.values(), ignore_index=True)
top500histories.shape

(8283, 36)

In [7]:
top500histories.to_csv('top500histories.csv')

In [8]:
top500histories.columns

Index(['assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element',
       'fixture', 'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'kickoff_time', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'gameweek', 'id_', 'sum_points', 'web_name',
       'next_week_points'],
      dtype='object')